In [6]:
#単一の CW レーザーを照射 ver.
using DelimitedFiles
using SharedArrays
using Random
using Distributed

core_num = 60 #number of cores used for multiprocessing
addprocs(core_num)
output = "um" #If you want to output calculation results micro meters, write um here.
save_path = "C:\\Users\\setol\\OneDrive\\ドキュメント\\simdata" 

@everywhere begin
    using SharedArrays
    using Distributed

    #parameters
    Tave = 298 #room temperature
    r_p = 250e-9 #radius of particle
    np = 1.44 #1.58 or 1.44 refractive index of particles
    nm = 1.33 #refractive index of midium
    ls_pow = 100e-3 #incident laser power(mW)
    ls_spot_x = 0 #x-position of the center of the laser spot
    ls_spot_y = 0 #y-position of the center of the laser spot
    w_l = 1064e-9# wavelength of laser
    NA = 0.9 #Numerical Aperture of objective lens
    FWHM = 1.22 * w_l / NA #full width at half maximum of the gaussian beam
    g_w = FWHM * sqrt(log(2) / 2) #Gaussian width at half maximum of the gaussian beam
    p_num = 10 #number of particles
    t = 60 #calculate time
    n = 100000 #steps
    ran = 20 #range of initial positions of pariticles(um)
    deltat = t / n #time width
    seed_num = 1 #seed value

    #constant
    N_A = 6.022 * 1e23 #Avogadro constant
    R = 8.314 #gas constant
    kb = R / N_A #Boltzman constant
    c = 3e8 #light speed

    #define functions
    #functions relating a BM
    #dynamic viscosity of water
    function eta(T)
        eta = 2.761 * exp(1713 / T) * 10 ^ (-6)
        return eta
    end
    #variance of the BM
    function sigma(T)
        sigma = sqrt((2 * kb * T * deltat) / (6 * pi * eta(T) * r_p))
        return sigma
    end
    #friction coefficient
    function gamma(T, a)
        gamma = 6 * pi *eta(T) * a
        return gamma
    end
    #generate random numbers
    function box_muller(u_1, u_2)
        X = sqrt.(-2 .* log.(u_1)) .* cos.(2 * pi .* u_2)
        Y = sqrt.(-2 .* log.(u_1)) .* sin.(2 * pi .* u_2)
        return X, Y
    end
    #optical gradient force of x-y plane
    #Here, we defined a optical gradient force of x-direction fx. If you calcuate a optical gradient force of y-direction fy, write ot(y, x).
    function fot(U0, x, y, r)
        fot = -4 * x * U0 * (r ^ -2) * exp(-2 * (x ^ 2 + y ^ 2) * r ^ (-2))
        return fot
    end
    #intensity of the laser
    function Int(power, rl)
        I = power / (pi * rl ^ 2)
        return I
    end
    #polarizability of particles
    function pol(a, np, nm)
        pol = 4 * pi * r_p ^ 3 * ((np ^ 2 - nm ^ 2) / (np ^ 2 + 2 * nm ^ 2))
        return pol
    end
    #depth of potential
    function U0(power, rl, a, np, nm)
        U0 = (Int(power, rl) * pol(a, np, nm) * nm) / c
        return U0
    end

    #generate displacements of BM
    function BM(p_num, T)
        xr = SharedArray{Float64}(p_num, n)
        yr = SharedArray{Float64}(p_num, n)
        @sync @distributed for i in 1:n
            u1 = rand(p_num, 1)
            u2 = rand(p_num, 1)
            X, Y = box_muller(u1, u2)
            xr[:, i] = X[:, 1]
            yr[:, i] = Y[:, 1]
        end
        sigmab = sigma(T)
        deltaxb = sigmab .* xr
        deltayb = sigmab .* yr
        return deltaxb, deltayb
    end

    #calculate a particle behavior
    function euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, g_w, np, nm)
        x_i = zeros((n+1)) #particle x-position
        y_i = zeros((n+1)) #particle y-position
        x_i[1] = x0[j, 1]#set a initial x-position
        y_i[1] = y0[j, 1]#set a initial y-position
        xb = deltaxb[j, :]
        yb = deltayb[j, :]
        gammaf = gamma(Tave, r_p)
        U0f = U0(ls_pow, g_w, r_p, np, nm)
        #Let's calculate a particle behavior
        for i in 1:n
            x_i[i+1] = x_i[i] + xb[i] + fot(U0f, x_i[i] - ls_spot_x, y_i[i] - ls_spot_y, g_w) * deltat / gammaf
            y_i[i+1] = y_i[i] + yb[i] + fot(U0f, y_i[i] - ls_spot_y, x_i[i] - ls_spot_x, g_w) * deltat / gammaf
        end
        return x_i, y_i
    end
    #define parallel calculations
    function euler_para(p_num, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, g_w, np, nm)
        resultx = SharedArray{Float64}(p_num, n+1)
        resulty = SharedArray{Float64}(p_num, n+1)
        @sync @distributed for j = 1:p_num
            resultx[j, :], resulty[j, :] = euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, g_w, np, nm)
        end

        if output == "um"
            resultx = resultx .* 1e6
            resulty = resulty .* 1e6
        end
        cd("$(save_path)")
        writedlm("x_$(ls_pow)mW_$(r_p)nm_sili2.txt", resultx, "\t")
        writedlm("y_$(ls_pow)mW_$(r_p)nm_sili2.txt", resulty, "\t")
    end

    #main
    function main(p_num, n, Tave, r_p, ls_pow, g_w, np, nm, ran)
        #generate displacements of Brrownian motion
        Random.seed!(seed_num)
        xb, yb = BM(p_num, Tave)
        #generate initial positionas
        x0 = rand(p_num, 1) .* 2 * -ran * 1e-6 .+ ran * 1e-6
        y0 = rand(p_num, 1) .* 2 * -ran * 1e-6 .+ ran * 1e-6
        euler_para(p_num, n, x0, y0, xb, yb, Tave, r_p, ls_pow, g_w, np, nm)
    end
end
@time main(p_num, n, Tave, r_p, ls_pow, g_w, np, nm, ran)

 91.348920 seconds (6.92 M allocations: 1013.730 MiB, 0.54% gc time, 2 lock conflicts, 0.36% compilation time)


In [7]:
# ガウシアンビームの光強度分布計算と可視化のみを実行するJuliaネイティブスクリプト

using Plots        # Plots.jlを使用 (事前インストールが必要)
using LinearAlgebra

# --- 環境設定 ---
# PNGファイルの保存先パス (適宜変更してください)
# 元のシミュレーションコードのパスを使用
save_path = "C:\\Users\\setol\\OneDrive\\ドキュメント\\simdata" 
# 事前にこのディレクトリが存在することを確認してください

# --- ビームおよび可視化パラメーター (元のコードから流用) ---
w_l = 1064e-9  # 波長 (m)
NA = 0.9       # 開口数
ls_spot_x = 0  # スポット中心 x
ls_spot_y = 0  # スポット中心 y

# ガウシアンビームの定義: w_g (Gaussian width)
FWHM = 1.22 * w_l / NA 
g_w = FWHM * sqrt(log(2) / 2) # Gaussian width at half maximum of the gaussian beam

# 可視化パラメータ
const GRID_SIZE = 200            # 描画グリッド解像度
const GW_UM = g_w * 1e6          # ガウシアン幅 (μm)
# 描画範囲をGaussian幅の約4倍に設定
const INTENSITY_RANGE_UM = 4.0 * GW_UM 

# ----------------------------------------------------------------

# --- ガウシアンビーム関連関数 ---

# 規格化されたガウシアンビームの強度（I/I_peak）
function Gaussian_Intensity_Normalized(x::Real, y::Real, g_w::Real, x_spot::Real, y_spot::Real)
    # ガウシアンビームの強度 I(r) は exp(-2 * r^2 / w^2) に比例します。
    # ここで w (Gaussian width) は g_w に相当
    r_sq = (x - x_spot)^2 + (y - y_spot)^2
    return exp(-2 * r_sq / g_w^2)
end


# 強度分布を計算し、グリッドデータと座標を返す関数
function calculate_intensity_grid(g_w, x_spot, y_spot, range_um, grid_size)
    max_r = range_um * 1e-6 # m単位

    # 座標グリッドを生成 (μm単位)
    coords_m = LinRange(-max_r, max_r, grid_size)
    coords_um = coords_m .* 1e6 
    intensity_grid = zeros(Float64, grid_size, grid_size)

    for i in 1:grid_size
        x = coords_m[i]
        for j in 1:grid_size
            y = coords_m[j]
            
            # 強度関数を呼び出し（規格化された相対強度）
            intensity_grid[i, j] = Gaussian_Intensity_Normalized(x, y, g_w, x_spot, y_spot)
        end
    end

    # Plots.jl の heatmap は (x, y, data) の順で、dataは行がX軸、列がY軸に対応するため、
    # i=X, j=Y に対応するよう計算結果をそのまま返す
    return intensity_grid, coords_um
end


# --- Plots.jl を使ってPNGを生成する関数 (Juliaネイティブ) ---
function plot_intensity_png(save_path, intensity_grid, coords_um, g_w_um)
    # GRバックエンドを使用
    gr() 

    output_filename = "Gaussian_intensity_visualizer.png"
    output_filepath = joinpath(save_path, output_filename)

    # 座標軸の範囲 (μm)
    x_ticks = round.([coords_um[1], 0.0, coords_um[end]], digits=2)
    y_ticks = x_ticks

    # タイトルにg_wの値を含める
    title_text = "Gaussian Beam Intensity Distribution (w = $(round(g_w_um, digits=3)) μm)"

    # 描画
    p = Plots.heatmap(
        coords_um, 
        coords_um, 
        intensity_grid,
        title=title_text,
        xlabel="X coordinate (μm)",
        ylabel="Y coordinate (μm)",
        color=:hot,
        colorbar_title="Normalized Intensity (a.u.)",
        aspect_ratio=:equal,
        xticks=x_ticks,
        yticks=y_ticks,
        size=(600, 500) # ウィンドウサイズ
    )
    
    # PNGファイルとして保存
    Plots.savefig(p, output_filepath)

    println("--- 強度分布PNG が $(output_filepath) に保存されました。 ---")
end


# --- メイン実行 ---

println("--- ガウシアンビーム強度分布のPNG画像生成を開始（Juliaネイティブ） ---")

# 1. 強度グリッドを計算
intensity_grid, coords_um = calculate_intensity_grid(
    g_w, ls_spot_x, ls_spot_y, INTENSITY_RANGE_UM, GRID_SIZE
)

# 2. Plots.jlを使ってPNG画像を生成
plot_intensity_png(save_path, intensity_grid, coords_um, GW_UM)

println("\n--- 可視化タスクが完了しました。 ---")


--- ガウシアンビーム強度分布のPNG画像生成を開始（Juliaネイティブ） ---
--- 強度分布PNG が C:\Users\setol\OneDrive\ドキュメント\simdata\Gaussian_intensity_visualizer.png に保存されました。 ---

--- 可視化タスクが完了しました。 ---


In [11]:
#単一の CW レーザーを照射 ver. (LGビームに換装)
using DelimitedFiles
using SharedArrays
using Random
using Distributed
using SpecialPolynomials # LGビームの計算のために追加

core_num = 60 #number of cores used for multiprocessing
addprocs(core_num)
output = "um" #If you want to output calculation results micro meters, write um here.
save_path = "C:\\Users\\setol\\OneDrive\\ドキュメント\\simdata"  

# --- LGビームパラメータ (デフォルト: p=0, l=0, ガウスビームに相当) ---
p_index = 0
l_index = 0
w0_beam = 0.225e-6 # ビームウェスト w0 (m)
# ---------------------------------------------------------------

@everywhere begin
    using SharedArrays
    using Distributed
    using SpecialPolynomials 

    #parameters
    Tave = 298 #room temperature
    r_p = 250e-9 #radius of particle
    np = 1.44 #1.58 or 1.44 refractive index of particles
    nm = 1.33 #refractive index of midium
    ls_pow = 1000e-3 #incident laser power(mW)
    ls_spot_x = 0 #x-position of the center of the laser spot
    ls_spot_y = 0 #y-position of the center of the laser spot
    w_l = 1064e-9# wavelength of laser
    NA = 0.9 #Numerical Aperture of objective lens
    # FWHM, g_w はLGビームの計算では不要なため使用しない
    p_num = 10 #number of particles
    t = 60 #calculate time
    n = 1200000 #steps
    ran = 20 #range of initial positions of pariticles(um)
    deltat = t / n #time width
    seed_num = 1 #seed value

    #constant
    N_A = 6.022 * 1e23 #Avogadro constant
    R = 8.314 #gas constant
    kb = R / N_A #Boltzman constant
    c = 3e8 #light speed

    #define functions
    #functions relating a BM
    #dynamic viscosity of water
    function eta(T)
        eta = 2.761 * exp(1713 / T) * 10 ^ (-6)
        return eta
    end
    #variance of the BM
    function sigma(T)
        # deltat, r_p は @everywhere スコープ内のグローバル変数に依存
        sigma = sqrt((2 * kb * T * deltat) / (6 * pi * eta(T) * r_p))
        return sigma
    end
    #friction coefficient
    function gamma(T, a)
        gamma = 6 * pi *eta(T) * a
        return gamma
    end
    #generate random numbers
    function box_muller(u_1, u_2)
        X = sqrt.(-2 .* log.(u_1)) .* cos.(2 * pi .* u_2)
        Y = sqrt.(-2 .* log.(u_1)) .* sin.(2 * pi .* u_2)
        return X, Y
    end
    
    # --- LGビーム関連関数 (勾配力計算用) ---
    function Rayleigh_Range(w0, lambda)
        return pi * w0^2 / lambda
    end

    function Laguerre_L(p::Int, l::Int, x::Real)
        l_non_negative = abs(l)
        L_poly = SpecialPolynomials.basis(SpecialPolynomials.Laguerre{Float64(l_non_negative)}, p)
        return L_poly(x)
    end

    function LG_Spatial_Intensity_Normalized(r, z, p, l, w0, lambda)
        zR = Rayleigh_Range(w0, lambda)
        current_wz = w0 * sqrt(1.0 + (z / zR)^2)
        x_arg = 2.0 * r^2 / current_wz^2
        l_abs = abs(l)

        if r <= 1.0e-15 && l_abs != 0
            return 0.0
        end

        radial_term_power = x_arg^l_abs
        laguerre_term_squared = (Laguerre_L(p, l, x_arg))^2

        return (w0 / current_wz)^2 * radial_term_power * laguerre_term_squared * exp(-x_arg)
    end

    function particle_polarizability(a, np, nm)
        epsilon_m = nm^2
        return 4.0 * pi * epsilon_m * a^3 * ((np^2 - nm^2) / (np^2 + 2.0 * nm^2))
    end
    
    # LGビームによる光勾配力計算 (光勾配力のみ)
    function LG_Gradient_Force(x_pos, y_pos, z_pos, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        r = sqrt(x_pos^2 + y_pos^2)
        phi = atan(y_pos, x_pos)
        alpha_particle = particle_polarizability(r_p, np, nm)
        
        I_scaling_factor = ls_pow / (pi * w0_beam^2)
        
        # --- 1. 勾配力 (F_r: 半径方向) のための数値微分 ---
        dr_step = 1.0e-10
        
        # p_idx, l_idx は Int であることを保証されているため、そのまま使用
        I_current = LG_Spatial_Intensity_Normalized(r, z_pos, p_idx, l_idx, w0_beam, w_l)
        
        r_plus_dr = r + dr_step
        
        I_r_plus_dr = LG_Spatial_Intensity_Normalized(r_plus_dr, z_pos, p_idx, l_idx, w0_beam, w_l)
        dI_dr = (I_r_plus_dr - I_current) / dr_step

        F_r = alpha_particle * dI_dr * I_scaling_factor
        
        # --- 2. デカルト座標への変換 ---
        Fx = F_r * cos(phi)
        Fy = F_r * sin(phi)

        return Fx, Fy, 0.0
    end
    # --- -------------------------------------------- ---

    #generate displacements of BM
    function BM(p_num, T)
        xr = SharedArray{Float64}(p_num, n)
        yr = SharedArray{Float64}(p_num, n)
        @sync @distributed for i in 1:n
            u1 = rand(p_num, 1)
            u2 = rand(p_num, 1)
            X, Y = box_muller(u1, u2)
            xr[:, i] = X[:, 1]
            yr[:, i] = Y[:, 1]
        end
        sigmab = sigma(T)
        deltaxb = sigmab .* xr
        deltayb = sigmab .* yr
        return deltaxb, deltayb
    end

    #calculate a particle behavior
    # 引数に p_idx, l_idx, w0_beam, w_l を追加
    function euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        x_i = zeros((n+1)) #particle x-position
        y_i = zeros((n+1)) #particle y-position
        x_i[1] = x0[j, 1]#set a initial x-position
        y_i[1] = y0[j, 1]#set a initial y-position
        xb = deltaxb[j, :]
        yb = deltayb[j, :]
        gammaf = gamma(Tave, r_p)
        
        #Let's calculate a particle behavior
        for i in 1:n
            # LGビームの勾配力を計算
            # p_idx, l_idx は Int であることが保証されているため、そのまま渡す
            Fx_grad, Fy_grad, _ = LG_Gradient_Force(x_i[i] - ls_spot_x, y_i[i] - ls_spot_y, 0.0, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)

            x_i[i+1] = x_i[i] + xb[i] + Fx_grad * deltat / gammaf
            y_i[i+1] = y_i[i] + yb[i] + Fy_grad * deltat / gammaf
        end
        return x_i, y_i
    end
    
    #define parallel calculations
    # 引数に LGパラメータを追加
    function euler_para(p_num, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        resultx = SharedArray{Float64}(p_num, n+1)
        resulty = SharedArray{Float64}(p_num, n+1)
        @sync @distributed for j = 1:p_num
            # LGパラメータを euler に渡す
            # ここで p_idx, l_idx は Int であることを保証されている
            resultx[j, :], resulty[j, :] = euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        end

        if output == "um"
            resultx = resultx .* 1e6
            resulty = resulty .* 1e6
        end
        
        # LGパラメータを含むファイル名に修正
        power_mW = Int(round(ls_pow * 1000))
        radius_nm = Int(round(r_p * 1e9))
        filename_base = "LG_p$(p_idx)_l$(l_idx)_$(power_mW)mW_$(radius_nm)nm"
        
        cd("$(save_path)")
        writedlm("x_$(filename_base).txt", resultx, "\t")
        writedlm("y_$(filename_base).txt", resulty, "\t")
    end

    #main
    # 引数に LGパラメータを追加
    function main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_idx::Int, l_idx::Int, w_l)
        #generate displacements of Brrownian motion
        Random.seed!(seed_num)
        xb, yb = BM(p_num, Tave)
        #generate initial positionas
        x0 = rand(p_num, 1) .* 2 * -ran * 1e-6 .+ ran * 1e-6
        y0 = rand(p_num, 1) .* 2 * -ran * 1e-6 .+ ran * 1e-6
        euler_para(p_num, n, x0, y0, xb, yb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
    end
end

# --- メイン実行 ---
# LGビームのパラメータを引数として渡す
@time main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_index, l_index, w_l)

# プロセスを終了
rmprocs(workers())

 65.855732 seconds (72.43 M allocations: 11.305 GiB, 3.92% gc time, 2 lock conflicts, 0.51% compilation time)


Task (done) @0x000002019021d560

In [19]:
#単一の CW レーザーを照射 ver. (熱運動ゼロ版)
using DelimitedFiles
using SharedArrays
using Random
using Distributed
using SpecialPolynomials # LGビームの計算のために追加

core_num = 60 #number of cores used for multiprocessing
addprocs(core_num)
output = "um" #If you want to output calculation results micro meters, write um here.
save_path = "C:\\Users\\setol\\OneDrive\\ドキュメント\\simdata" 

# --- LGビームパラメータ (p=0, l=0, ガウスビームに相当) ---
p_index = 0
l_index = 0
w0_beam = 0.225e-6 # ビームウェスト w0 (m)
# ---------------------------------------------------------------

@everywhere begin
    using SharedArrays
    using Distributed
    using SpecialPolynomials 

    #parameters
    Tave = 0 #room temperature (摩擦係数の計算にのみ使用)
    r_p = 100e-9 #radius of particle
    np = 1.44 #1.58 or 1.44 refractive index of particles
    nm = 1.33 #refractive index of midium
    ls_pow = 1000000e-3 #incident laser power(mW)
    ls_spot_x = 0 #x-position of the center of the laser spot
    ls_spot_y = 0 #y-position of the center of the laser spot
    w_l = 1064e-9# wavelength of laser
    NA = 0.9 #Numerical Aperture of objective lens
    p_num = 10 #number of particles
    t = 10 #calculate time
    n = 1200000 #steps
    ran = 20 #range of initial positions of pariticles(um)
    deltat = t / n #time width
    seed_num = 1 #seed value

    #constant
    N_A = 6.022 * 1e23 #Avogadro constant
    R = 8.314 #gas constant
    kb = R / N_A #Boltzman constant
    c = 3e8 #light speed

    #define functions
    #functions relating a BM
    #dynamic viscosity of water (摩擦係数の計算に必要なので残す)
    function eta(T)
        eta = 2.761 * exp(1713 / T) * 10 ^ (-6)
        return eta
    end
    #friction coefficient (残す)
    function gamma(T, a)
        gamma = 6 * pi *eta(T) * a
        return gamma
    end
    # --- sigma, box_muller は熱運動削除のため削除 ---
    
    # --- LGビーム関連関数 (勾配力計算用) ---
    function Rayleigh_Range(w0, lambda)
        return pi * w0^2 / lambda
    end

    function Laguerre_L(p::Int, l::Int, x::Real)
        l_non_negative = abs(l)
        L_poly = SpecialPolynomials.basis(SpecialPolynomials.Laguerre{Float64(l_non_negative)}, p)
        return L_poly(x)
    end

    function LG_Spatial_Intensity_Normalized(r, z, p, l, w0, lambda)
        zR = Rayleigh_Range(w0, lambda)
        current_wz = w0 * sqrt(1.0 + (z / zR)^2)
        x_arg = 2.0 * r^2 / current_wz^2
        l_abs = abs(l)

        if r <= 1.0e-15 && l_abs != 0
            return 0.0
        end

        radial_term_power = x_arg^l_abs
        laguerre_term_squared = (Laguerre_L(p, l, x_arg))^2

        return (w0 / current_wz)^2 * radial_term_power * laguerre_term_squared * exp(-x_arg)
    end

    function particle_polarizability(a, np, nm)
        epsilon_m = nm^2
        return 4.0 * pi * epsilon_m * a^3 * ((np^2 - nm^2) / (np^2 + 2.0 * nm^2))
    end
    
    # LGビームによる光勾配力計算 (光勾配力のみ)
    function LG_Gradient_Force(x_pos, y_pos, z_pos, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        r = sqrt(x_pos^2 + y_pos^2)
        phi = atan(y_pos, x_pos)
        alpha_particle = particle_polarizability(r_p, np, nm)
        
        I_scaling_factor = ls_pow / (pi * w0_beam^2)
        
        # --- 1. 勾配力 (F_r: 半径方向) のための数値微分 ---
        dr_step = 1.0e-10
        
        I_current = LG_Spatial_Intensity_Normalized(r, z_pos, p_idx, l_idx, w0_beam, w_l)
        
        r_plus_dr = r + dr_step
        
        I_r_plus_dr = LG_Spatial_Intensity_Normalized(r_plus_dr, z_pos, p_idx, l_idx, w0_beam, w_l)
        dI_dr = (I_r_plus_dr - I_current) / dr_step

        F_r = alpha_particle * dI_dr * I_scaling_factor
        
        # --- 2. デカルト座標への変換 ---
        Fx = F_r * cos(phi)
        Fy = F_r * sin(phi)

        return Fx, Fy, 0.0
    end
    # --- -------------------------------------------- ---

    # --- BM関数を削除し、空のBM関数を定義 (メイン関数がエラーにならないように) ---
    function BM(p_num, T)
        # 熱運動ゼロなので、空の行列を返す
        deltaxb = SharedArray{Float64}(p_num, n)
        deltayb = SharedArray{Float64}(p_num, n)
        deltaxb .= 0.0
        deltayb .= 0.0
        return deltaxb, deltayb
    end
    # --- ---------------------------------------------------------------------- ---

    #calculate a particle behavior
    # 引数に p_idx, l_idx, w0_beam, w_l を追加
    function euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        x_i = zeros((n+1)) #particle x-position
        y_i = zeros((n+1)) #particle y-position
        x_i[1] = x0[j, 1]#set a initial x-position
        y_i[1] = y0[j, 1]#set a initial y-position
        # xb = deltaxb[j, :] <--- 削除
        # yb = deltayb[j, :] <--- 削除
        gammaf = gamma(Tave, r_p)
        
        #Let's calculate a particle behavior
        for i in 1:n
            # LGビームの勾配力を計算
            Fx_grad, Fy_grad, _ = LG_Gradient_Force(x_i[i] - ls_spot_x, y_i[i] - ls_spot_y, 0.0, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)

            # 位置更新: 熱運動の項 (xb[i]) を削除
            x_i[i+1] = x_i[i] + Fx_grad * deltat / gammaf
            y_i[i+1] = y_i[i] + Fy_grad * deltat / gammaf
        end
        return x_i, y_i
    end
    
    #define parallel calculations
    # 引数に LGパラメータを追加
    function euler_para(p_num, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        resultx = SharedArray{Float64}(p_num, n+1)
        resulty = SharedArray{Float64}(p_num, n+1)
        @sync @distributed for j = 1:p_num
            # LGパラメータを euler に渡す
            resultx[j, :], resulty[j, :] = euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        end

        if output == "um"
            resultx = resultx .* 1e6
            resulty = resulty .* 1e6
        end
        
        # LGパラメータを含むファイル名に修正
        power_mW = Int(round(ls_pow * 1000))
        radius_nm = Int(round(r_p * 1e9))
        filename_base = "LG_p$(p_idx)_l$(l_idx)_$(power_mW)mW_$(radius_nm)nm_noBM" # ファイル名に noBM を追加
        
        cd("$(save_path)")
        writedlm("x_$(filename_base).txt", resultx, "\t")
        writedlm("y_$(filename_base).txt", resulty, "\t")
    end

    #main
    # 引数に LGパラメータを追加
    function main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_idx::Int, l_idx::Int, w_l)
        # generate displacements of Brrownian motion -> BM関数は空の行列を返す
        Random.seed!(seed_num)
        xb, yb = BM(p_num, Tave)
        #generate initial positionas
        x0 = rand(p_num, 1) .* 2 * -ran * 1e-6 .+ ran * 1e-6
        y0 = rand(p_num, 1) .* 2 * -ran * 1e-6 .+ ran * 1e-6
        euler_para(p_num, n, x0, y0, xb, yb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
    end
end

# --- メイン実行 ---
# LGビームのパラメータを引数として渡す
@time main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_index, l_index, w_l)

# プロセスを終了
rmprocs(workers())


 60.121066 seconds (72.32 M allocations: 11.120 GiB, 4.20% gc time, 0.49% compilation time)


Task (done) @0x0000020191685560

In [1]:
#単一の CW レーザーを照射 ver. (熱運動ゼロ・単一粒子)
using DelimitedFiles
using SharedArrays
using Random
using Distributed
using SpecialPolynomials # LGビームの計算のために追加

core_num = 24#number of cores used for multiprocessing
addprocs(core_num)
output = "um" #If you want to output calculation results micro meters, write um here.
save_path = "C:\\Users\\setol\\Documents\\sim_data"


# --- LGビームパラメータ (p=0, l=0, ガウスビームに相当) ---
p_index = 0
l_index = 1
w0_beam = 0.225e-6 # ビームウェスト w0 (m)
# ---------------------------------------------------------------

@everywhere begin
    using SharedArrays
    using Distributed
    using SpecialPolynomials 

    #parameters
    Tave = 0 #room temperature (熱運動ゼロのため、0Kに設定)
    r_p = 100e-9 #radius of particle
    np = 1.44 #1.58 or 1.44 refractive index of particles
    nm = 1.33 #refractive index of midium
    ls_pow = 1000e-3 #incident laser power(mW)
    ls_spot_x = 0 #x-position of the center of the laser spot
    ls_spot_y = 0 #y-position of the center of the laser spot
    w_l = 1064e-9# wavelength of laser
    NA = 0.9 #Numerical Aperture of objective lens
    p_num = 1 #number of particles <--- 1に固定
    t = 10 #calculate time
    n = 1200000 #steps
    ran = 20 #range of initial positions of pariticles(um)
    deltat = t / n #time width
    seed_num = 1 #seed value

    #constant
    N_A = 6.022 * 1e23 #Avogadro constant
    R = 8.314 #gas constant
    kb = R / N_A #Boltzman constant
    c = 3e8 #light speed

    #define functions
    #functions relating a BM
    #dynamic viscosity of water (摩擦係数の計算に必要なので残す)
    function eta(T)
        # T=0Kの場合は0除算を避けるため、Tave=298Kでの値を使用 (物理的な摩擦係数のみを維持)
        # T=0のとき、exp(1713/T)は無限大になるため、室温での粘性率を使用
        if T == 0
            T_ref = 298.0
        else
            T_ref = T
        end
        eta = 2.761 * exp(1713 / T_ref) * 10 ^ (-6)
        return eta
    end
    #friction coefficient (残す)
    function gamma(T, a)
        gamma = 6 * pi *eta(T) * a
        return gamma
    end
    # --- BM関数は空の行列を返す ---
    function BM(p_num, T)
        deltaxb = SharedArray{Float64}(p_num, n)
        deltayb = SharedArray{Float64}(p_num, n)
        deltaxb .= 0.0
        deltayb .= 0.0
        return deltaxb, deltayb
    end
    # --- LGビーム関連関数は省略 ---
    # ... (LG_Gradient_Forceまで全て省略) ...
    function Rayleigh_Range(w0, lambda)
        return pi * w0^2 / lambda
    end

    function Laguerre_L(p::Int, l::Int, x::Real)
        l_non_negative = abs(l)
        L_poly = SpecialPolynomials.basis(SpecialPolynomials.Laguerre{Float64(l_non_negative)}, p)
        return L_poly(x)
    end

    function LG_Spatial_Intensity_Normalized(r, z, p, l, w0, lambda)
        zR = Rayleigh_Range(w0, lambda)
        current_wz = w0 * sqrt(1.0 + (z / zR)^2)
        x_arg = 2.0 * r^2 / current_wz^2
        l_abs = abs(l)

        if r <= 1.0e-15 && l_abs != 0
            return 0.0
        end

        radial_term_power = x_arg^l_abs
        laguerre_term_squared = (Laguerre_L(p, l, x_arg))^2

        return (w0 / current_wz)^2 * radial_term_power * laguerre_term_squared * exp(-x_arg)
    end

    function particle_polarizability(a, np, nm)
        epsilon_m = nm^2
        return 4.0 * pi * epsilon_m * a^3 * ((np^2 - nm^2) / (np^2 + 2.0 * nm^2))
    end
    
    function LG_Gradient_Force(x_pos, y_pos, z_pos, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        r = sqrt(x_pos^2 + y_pos^2)
        phi = atan(y_pos, x_pos)
        alpha_particle = particle_polarizability(r_p, np, nm)
        
        I_scaling_factor = ls_pow / (pi * w0_beam^2)
        
        dr_step = 1.0e-10
        
        I_current = LG_Spatial_Intensity_Normalized(r, z_pos, p_idx, l_idx, w0_beam, w_l)
        
        r_plus_dr = r + dr_step
        
        I_r_plus_dr = LG_Spatial_Intensity_Normalized(r_plus_dr, z_pos, p_idx, l_idx, w0_beam, w_l)
        dI_dr = (I_r_plus_dr - I_current) / dr_step

        F_r = alpha_particle * dI_dr * I_scaling_factor
        Fx = F_r * cos(phi)
        Fy = F_r * sin(phi)
        return Fx, Fy, 0.0
    end
    # --- ---

    #calculate a particle behavior
    # 引数に p_idx, l_idx, w0_beam, w_l を追加
    function euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        x_i = zeros((n+1)) #particle x-position
        y_i = zeros((n+1)) #particle y-position
        x_i[1] = x0[j, 1]#set a initial x-position
        y_i[1] = y0[j, 1]#set a initial y-position
        # 熱運動項はゼロ

        gammaf = gamma(Tave, r_p)
        
        #Let's calculate a particle behavior
        for i in 1:n
            # LGビームの勾配力を計算
            Fx_grad, Fy_grad, _ = LG_Gradient_Force(x_i[i] - ls_spot_x, y_i[i] - ls_spot_y, 0.0, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)

            # 位置更新: 熱運動の項を削除
            x_i[i+1] = x_i[i] + Fx_grad * deltat / gammaf
            y_i[i+1] = y_i[i] + Fy_grad * deltat / gammaf
        end
        return x_i, y_i
    end
    
    #define parallel calculations
    # 引数に LGパラメータを追加
    function euler_para(p_num, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        resultx = SharedArray{Float64}(p_num, n+1)
        resulty = SharedArray{Float64}(p_num, n+1)
        @sync @distributed for j = 1:p_num
            # LGパラメータを euler に渡す
            resultx[j, :], resulty[j, :] = euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        end

        if output == "um"
            resultx = resultx .* 1e6
            resulty = resulty .* 1e6
        end
        
        # LGパラメータを含むファイル名に修正
        power_mW = Int(round(ls_pow * 1000))
        radius_nm = Int(round(r_p * 1e9))
        filename_base = "LG_p$(p_idx)_l$(l_idx)_$(power_mW)mW_$(radius_nm)nm_noBM_single" # single を追加
        
        cd("$(save_path)")
        writedlm("x_$(filename_base).txt", resultx, "\t")
        writedlm("y_$(filename_base).txt", resulty, "\t")
    end

    #main
    # 引数に LGパラメータを追加
    function main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_idx::Int, l_idx::Int, w_l)
        # generate displacements of Brrownian motion -> BM関数は空の行列を返す
        Random.seed!(seed_num)
        xb, yb = BM(p_num, Tave)
        #generate initial positionas -> x=0, y=0に固定
        x0 = zeros(p_num, 1) # x=0に固定
        y0 = zeros(p_num, 1) # y=0に固定
        euler_para(p_num, n, x0, y0, xb, yb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
    end
end

# --- メイン実行 ---
# LGビームのパラメータを引数として渡す
@time main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_index, l_index, w_l)

# プロセスを終了
rmprocs(workers())


 20.825569 seconds (11.59 M allocations: 1.280 GiB, 2.26% gc time, 2 lock conflicts, 8.10% compilation time)


Task (done) @0x0000011c3ef089c0

In [16]:
#単一の CW レーザーを照射 ver. (熱運動ゼロ・単一粒子)
using DelimitedFiles
using SharedArrays
using Random
using Distributed
using SpecialPolynomials # LGビームの計算のために追加

core_num = 24#number of cores used for multiprocessing
addprocs(core_num)
output = "um" #If you want to output calculation results micro meters, write um here.
save_path = "C:\\Users\\setol\\Documents\\sim_data"


# --- LGビームパラメータ (p=0, l=1) ---
p_index = 0
l_index = 1
w0_beam = 0.5e-6 # ビームウェスト w0 (m)
# ---------------------------------------------------------------

@everywhere begin
    using SharedArrays
    using Distributed
    using SpecialPolynomials 

    #parameters
    Tave = 0 #room temperature (熱運動ゼロのため、0Kに設定)
    r_p = 100e-9 #radius of particle
    np = 1.44 #1.58 or 1.44 refractive index of particles
    nm = 1.33 #refractive index of midium
    ls_pow = 1000e-3 #incident laser power(mW)
    ls_spot_x = 0 #x-position of the center of the laser spot
    ls_spot_y = 0 #y-position of the center of the laser spot
    w_l = 1064e-9# wavelength of laser
    NA = 0.9 #Numerical Aperture of objective lens
    p_num = 1 #number of particles <--- 1に固定
    t = 60 #calculate time
    n = 1200000 #steps
    ran = 20 #range of initial positions of pariticles(um)
    deltat = t / n #time width
    seed_num = 1 #seed value

    #constant
    N_A = 6.022 * 1e23 #Avogadro constant
    R = 8.314 #gas constant
    kb = R / N_A #Boltzman constant
    c = 3e8 #light speed

    #define functions
    #functions relating a BM
    #dynamic viscosity of water (摩擦係数の計算に必要なので残す)
    function eta(T)
        # T=0Kの場合は0除算を避けるため、Tave=298Kでの値を使用 (物理的な摩擦係数のみを維持)
        # T=0のとき、exp(1713/T)は無限大になるため、室温での粘性率を使用
        if T == 0
            T_ref = 298.0
        else
            T_ref = T
        end
        eta = 2.761 * exp(1713 / T_ref) * 10 ^ (-6)
        return eta
    end
    #friction coefficient (残す)
    function gamma(T, a)
        gamma = 6 * pi *eta(T) * a
        return gamma
    end
    # --- BM関数は空の行列を返す (熱運動ゼロ) ---
    function BM(p_num, T)
        deltaxb = SharedArray{Float64}(p_num, n)
        deltayb = SharedArray{Float64}(p_num, n)
        deltaxb .= 0.0
        deltayb .= 0.0
        return deltaxb, deltayb
    end
    
    # --- LGビーム関連関数 ---
    function Rayleigh_Range(w0, lambda)
        return pi * w0^2 / lambda
    end

    function Laguerre_L(p::Int, l::Int, x::Real)
        l_non_negative = abs(l)
        L_poly = SpecialPolynomials.basis(SpecialPolynomials.Laguerre{Float64(l_non_negative)}, p)
        return L_poly(x)
    end

    function LG_Spatial_Intensity_Normalized(r, z, p, l, w0, lambda)
        zR = Rayleigh_Range(w0, lambda)
        current_wz = w0 * sqrt(1.0 + (z / zR)^2)
        x_arg = 2.0 * r^2 / current_wz^2
        l_abs = abs(l)

        if r <= 1.0e-15 && l_abs != 0
            return 0.0
        end

        radial_term_power = x_arg^l_abs
        laguerre_term_squared = (Laguerre_L(p, l, x_arg))^2

        return (w0 / current_wz)^2 * radial_term_power * laguerre_term_squared * exp(-x_arg)
    end

    function particle_polarizability(a, np, nm)
        epsilon_m = nm^2
        return 4.0 * pi * epsilon_m * a^3 * ((np^2 - nm^2) / (np^2 + 2.0 * nm^2))
    end
    
    function LG_Gradient_Force(x_pos, y_pos, z_pos, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        r = sqrt(x_pos^2 + y_pos^2)
        phi = atan(y_pos, x_pos)
        alpha_particle = particle_polarizability(r_p, np, nm)
        
        I_scaling_factor = ls_pow / (pi * w0_beam^2)
        
        dr_step = 1.0e-10
        
        I_current = LG_Spatial_Intensity_Normalized(r, z_pos, p_idx, l_idx, w0_beam, w_l)
        
        r_plus_dr = r + dr_step
        
        I_r_plus_dr = LG_Spatial_Intensity_Normalized(r_plus_dr, z_pos, p_idx, l_idx, w0_beam, w_l)
        dI_dr = (I_r_plus_dr - I_current) / dr_step

        F_r = alpha_particle * dI_dr * I_scaling_factor
        Fx = F_r * cos(phi)
        Fy = F_r * sin(phi)
        return Fx, Fy, 0.0
    end
    # --- ---

    #calculate a particle behavior
    # 引数に p_idx, l_idx, w0_beam, w_l を追加
    function euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        x_i = zeros((n+1)) #particle x-position
        y_i = zeros((n+1)) #particle y-position
        x_i[1] = x0[j, 1]#set a initial x-position
        y_i[1] = y0[j, 1]#set a initial y-position
        # 熱運動項はゼロ

        gammaf = gamma(Tave, r_p)
        
        #Let's calculate a particle behavior
        for i in 1:n
            # LGビームの勾配力を計算
            Fx_grad, Fy_grad, _ = LG_Gradient_Force(x_i[i] - ls_spot_x, y_i[i] - ls_spot_y, 0.0, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)

            # 位置更新: 熱運動の項を削除
            x_i[i+1] = x_i[i] + Fx_grad * deltat / gammaf
            y_i[i+1] = y_i[i] + Fy_grad * deltat / gammaf
        end
        return x_i, y_i
    end
    
    #define parallel calculations
    # 引数に LGパラメータを追加
    function euler_para(p_num, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        resultx = SharedArray{Float64}(p_num, n+1)
        resulty = SharedArray{Float64}(p_num, n+1)
        @sync @distributed for j = 1:p_num
            # LGパラメータを euler に渡す
            resultx[j, :], resulty[j, :] = euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        end

        if output == "um"
            resultx = resultx .* 1e6
            resulty = resulty .* 1e6
        end
        
        # LGパラメータを含むファイル名に修正
        power_mW = Int(round(ls_pow * 1000))
        radius_nm = Int(round(r_p * 1e9))
        filename_base = "LG_p$(p_idx)_l$(l_idx)_$(power_mW)mW_$(radius_nm)nm_noBM_single" # single を追加
        
        cd("$(save_path)")
        writedlm("x_$(filename_base).txt", resultx, "\t")
        writedlm("y_$(filename_base).txt", resulty, "\t")
    end

    #main
    # 引数に LGパラメータを追加
   function main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_idx::Int, l_idx::Int, w_l)
        # generate displacements of Brrownian motion -> BM関数は空の行列を返す
        Random.seed!(seed_num)
        xb, yb = BM(p_num, Tave)
        
        # --- 変更箇所: 初期位置を光渦リング上 (R_trap) に設定 ---
        # r_max (R_trap) は強度極大半径 w0 / sqrt(2) に相当
        r_max = w0_beam / sqrt(2.0)
        
        # 粒子を光渦のリング上 (R_trap = r_max) からスタート
        initial_r_trap = r_max * 1.01 # r_max * 1.1 を r_max に変更
        
        x0 = ones(p_num, 1) .* initial_r_trap # x=R_trap に固定
        y0 = zeros(p_num, 1) # y=0 に固定 (x軸上)
        # ---------------------------------------------------------------------

        euler_para(p_num, n, x0, y0, xb, yb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
    end
end

# --- メイン実行 ---
# LGビームのパラメータを引数として渡す
@time main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_index, l_index, w_l)

# プロセスを終了
rmprocs(workers())

 19.642790 seconds (7.34 M allocations: 1.072 GiB, 2.15% gc time, 0.55% compilation time)


Task (done) @0x0000011cba086100

In [19]:
#単一の CW レーザーを照射 ver. (熱運動ゼロ・単一粒子)
using DelimitedFiles
using SharedArrays
using Random
using Distributed
using SpecialPolynomials # LGビームの計算のために追加

core_num = 24#number of cores used for multiprocessing
addprocs(core_num)
output = "um" #If you want to output calculation results micro meters, write um here.
save_path = "C:\\Users\\setol\\Documents\\sim_data"


# --- LGビームパラメータ (p=0, l=1) ---
p_index = 0
l_index = 1
w0_beam = 0.5e-6 # ビームウェスト w0 (m)
# ---------------------------------------------------------------

@everywhere begin
    using SharedArrays
    using Distributed
    using SpecialPolynomials 

    #parameters
    Tave = 0 #room temperature (熱運動ゼロのため、0Kに設定)
    r_p = 100e-9 #radius of particle
    np = 1.44 #1.58 or 1.44 refractive index of particles
    nm = 1.33 #refractive index of midium
    ls_pow = 1000e-3 #incident laser power(mW)
    ls_spot_x = 0 #x-position of the center of the laser spot
    ls_spot_y = 0 #y-position of the center of the laser spot
    w_l = 1064e-9# wavelength of laser
    NA = 0.9 #Numerical Aperture of objective lens
    p_num = 1 #number of particles <--- 1に固定
    t = 30 #calculate time
    n = 1200000 #steps <-- 安定性向上のため、1,200,000から12,000,000に10倍増加
    ran = 20 #range of initial positions of pariticles(um)
    deltat = t / n #time width
    seed_num = 1 #seed value

    #constant
    N_A = 6.022 * 1e23 #Avogadro constant
    R = 8.314 #gas constant
    kb = R / N_A #Boltzman constant
    c = 3e8 #light speed

    #define functions
    #functions relating a BM
    #dynamic viscosity of water (摩擦係数の計算に必要なので残す)
    function eta(T)
        # T=0Kの場合は0除算を避けるため、Tave=298Kでの値を使用 (物理的な摩擦係数のみを維持)
        # T=0のとき、exp(1713/T)は無限大になるため、室温での粘性率を使用
        if T == 0
            T_ref = 298.0
        else
            T_ref = T
        end
        eta = 2.761 * exp(1713 / T_ref) * 10 ^ (-6)
        return eta
    end
    #friction coefficient (残す)
    function gamma(T, a)
        gamma = 6 * pi *eta(T) * a
        return gamma
    end
    # --- BM関数は空の行列を返す (熱運動ゼロ) ---
    function BM(p_num, T)
        deltaxb = SharedArray{Float64}(p_num, n)
        deltayb = SharedArray{Float64}(p_num, n)
        deltaxb .= 0.0
        deltayb .= 0.0
        return deltaxb, deltayb
    end
    
    # --- LGビーム関連関数 ---
    function Rayleigh_Range(w0, lambda)
        return pi * w0^2 / lambda
    end

    function Laguerre_L(p::Int, l::Int, x::Real)
        l_non_negative = abs(l)
        L_poly = SpecialPolynomials.basis(SpecialPolynomials.Laguerre{Float64(l_non_negative)}, p)
        return L_poly(x)
    end

    function LG_Spatial_Intensity_Normalized(r, z, p, l, w0, lambda)
        zR = Rayleigh_Range(w0, lambda)
        current_wz = w0 * sqrt(1.0 + (z / zR)^2)
        x_arg = 2.0 * r^2 / current_wz^2
        l_abs = abs(l)

        if r <= 1.0e-15 && l_abs != 0
            return 0.0
        end

        radial_term_power = x_arg^l_abs
        laguerre_term_squared = (Laguerre_L(p, l, x_arg))^2

        return (w0 / current_wz)^2 * radial_term_power * laguerre_term_squared * exp(-x_arg)
    end

    function particle_polarizability(a, np, nm)
        epsilon_m = nm^2
        return 4.0 * pi * epsilon_m * a^3 * ((np^2 - nm^2) / (np^2 + 2.0 * nm^2))
    end
    
    function LG_Gradient_Force(x_pos, y_pos, z_pos, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        r = sqrt(x_pos^2 + y_pos^2)
        phi = atan(y_pos, x_pos)
        alpha_particle = particle_polarizability(r_p, np, nm)
        
        I_scaling_factor = ls_pow / (pi * w0_beam^2)
        
        dr_step = 1.0e-10
        
        I_current = LG_Spatial_Intensity_Normalized(r, z_pos, p_idx, l_idx, w0_beam, w_l)
        
        r_plus_dr = r + dr_step
        
        I_r_plus_dr = LG_Spatial_Intensity_Normalized(r_plus_dr, z_pos, p_idx, l_idx, w0_beam, w_l)
        dI_dr = (I_r_plus_dr - I_current) / dr_step

        # !! 修正: 勾配力は I の高い方向(勾配方向)に働くため、符号は正 (前回誤って追加した負の符号を削除) !!
        # F_r は dI/dr と同じ符号を持つ必要がある
        F_r = alpha_particle * dI_dr * I_scaling_factor 
        
        Fx = F_r * cos(phi)
        Fy = F_r * sin(phi)
        return Fx, Fy, 0.0
    end
    # --- ---

    #calculate a particle behavior
    # 引数に p_idx, l_idx, w0_beam, w_l を追加
    function euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        x_i = zeros((n+1)) #particle x-position
        y_i = zeros((n+1)) #particle y-position
        x_i[1] = x0[j, 1]#set a initial x-position
        y_i[1] = y0[j, 1]#set a initial y-position
        # 熱運動項はゼロ

        gammaf = gamma(Tave, r_p)
        
        #Let's calculate a particle behavior
        for i in 1:n
            # LGビームの勾配力を計算
            Fx_grad, Fy_grad, _ = LG_Gradient_Force(x_i[i] - ls_spot_x, y_i[i] - ls_spot_y, 0.0, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)

            # 位置更新: 熱運動の項を削除
            x_i[i+1] = x_i[i] + Fx_grad * deltat / gammaf
            y_i[i+1] = y_i[i] + Fy_grad * deltat / gammaf
        end
        return x_i, y_i
    end
    
    #define parallel calculations
    # 引数に LGパラメータを追加
    function euler_para(p_num, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        resultx = SharedArray{Float64}(p_num, n+1)
        resulty = SharedArray{Float64}(p_num, n+1)
        @sync @distributed for j = 1:p_num
            # LGパラメータを euler に渡す
            resultx[j, :], resulty[j, :] = euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        end

        if output == "um"
            resultx = resultx .* 1e6
            resulty = resulty .* 1e6
        end
        
        # LGパラメータを含むファイル名に修正
        power_mW = Int(round(ls_pow * 1000))
        radius_nm = Int(round(r_p * 1e9))
        filename_base = "LG_p$(p_idx)_l$(l_idx)_$(power_mW)mW_$(radius_nm)nm_noBM_single" # single を追加
        
        cd("$(save_path)")
        writedlm("x_$(filename_base).txt", resultx, "\t")
        writedlm("y_$(filename_base).txt", resulty, "\t")
    end

    #main
    # 引数に LGパラメータを追加
   function main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_idx::Int, l_idx::Int, w_l)
        # generate displacements of Brrownian motion -> BM関数は空の行列を返す
        Random.seed!(seed_num)
        xb, yb = BM(p_num, Tave)
        
        # --- 修正箇所: 初期位置を光渦リングより少し外側 (r_max * 1.1) に再設定 ---
        # トラッピングの挙動を確認するため、外側からスタートするように修正
        r_max = w0_beam / sqrt(2.0)
        
        # 粒子をトラップされることを確認するため、理論上の強度極大半径 r_max よりも10%外側からスタート
        initial_r_trap = r_max * 1.0 # R_trap * 1.1 に再変更
        
        x0 = ones(p_num, 1) .* initial_r_trap # x=r_max*1.1 に固定
        y0 = zeros(p_num, 1) # y=0 に固定 (x軸上)
        # ---------------------------------------------------------------------

        euler_para(p_num, n, x0, y0, xb, yb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
    end
end

# --- メイン実行 ---
# LGビームのパラメータを引数として渡す
@time main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_index, l_index, w_l)

# プロセスを終了
rmprocs(workers())

 19.511807 seconds (7.34 M allocations: 1.072 GiB, 2.11% gc time, 0.56% compilation time)


Task (done) @0x0000011c3eaea100

In [ ]:
    # --- 追加: ランダムな初期位置を生成する関数 ---
    function generate_initial_position(p_num, w0_beam, ls_pow, np, nm, w_l)
        r_max = w0_beam / sqrt(2.0)  # LG01モードの強度極大の半径 (z=0)
        r = zeros(p_num)
        theta = zeros(p_num)

        # 初期位置の動径距離 r を、半径 r_max の円盤内で均一にランダム化
        for i in 1:p_num
            # r^2 が [0, r_max^2] で均一になるように (面積に比例するランダムサンプリング)
            r[i] = sqrt(rand()) * r_max
            theta[i] = 2 * pi * rand()  # 角度は [0, 2π] の範囲でランダム
        end

        # x, y の位置を決定
        x0 = r .* cos.(theta)
        y0 = r .* sin.(theta)

        return x0, y0
    end

In [27]:
#2025/11/20完成版
#単一の CW レーザーを照射 ver. (熱運動ゼロ・単一粒子)
using DelimitedFiles
using SharedArrays
using Random
using Distributed
using SpecialPolynomials # LGビームの計算のために追加
# Printf はデバッグログの削除に伴い不要になりました


core_num = 24#number of cores used for multiprocessing
addprocs(core_num)
output = "um" #If you want to output calculation results micro meters, write um here.
save_path = "C:\\Users\\setol\\Documents\\sim_data"


# --- LGビームパラメータ (p=0, l=1) ---
p_index = 0
l_index = 1
w0_beam = 1.0e-6 # ビームウェスト w0 (m)
# ---------------------------------------------------------------

# --- ワーカープロセスでのパッケージの明示的なロード ---
# Printf は不要になったため削除
@everywhere using SharedArrays
@everywhere using Distributed
@everywhere using SpecialPolynomials 
@everywhere using Random 
# --------------------------------------------------------

@everywhere begin
    # Note: 必要なパッケージは既に @everywhere using でロード済み

    #parameters
    Tave = 298 #room temperature (熱運動ゼロのため、0Kに設定)
    r_p = 100e-9 #radius of particle
    np = 1.44 #1.58 or 1.44 refractive index of particles
    nm = 1.33 #refractive index of midium
    ls_pow = 1000e-3 #incident laser power(mW)
    ls_spot_x = 0 #x-position of the center of the laser spot
    ls_spot_y = 0 #y-position of the center of the laser spot
    w_l = 1064e-9# wavelength of laser
    NA = 0.9 #Numerical Aperture of objective lens
    p_num = 10 #number of particles <--- 1に固定
    t = 60 #calculate time
    n = 120000 #steps <-- 安定性向上のため、1,200,000から12,000,000に10倍増加
    ran = 20 #range of initial positions of pariticles(um)
    deltat = t / n #time width
    seed_num = 1 #seed value

    #constant
    N_A = 6.022 * 1e23 #Avogadro constant
    R = 8.314 #gas constant
    kb = R / N_A #Boltzman constant
    c = 3e8 #light speed

    #define functions
    #functions relating a BM
    #dynamic viscosity of water (摩擦係数の計算に必要なので残す)
    function eta(T)
        # T=0Kの場合は0除算を避けるため、Tave=298Kでの値を使用 (物理的な摩擦係数のみを維持)
        # T=0のとき、exp(1713/T)は無限大になるため、室温での粘性率を使用
        if T == 0
            T_ref = 298.0
        else
            T_ref = T
        end
        eta = 2.761 * exp(1713 / T_ref) * 10 ^ (-6)
        return eta
    end
    #friction coefficient (残す)
    function gamma(T, a)
        gamma = 6 * pi *eta(T) * a
        return gamma
    end
    # --- BM関数は空の行列を返す (熱運動ゼロ) ---
    function BM(p_num, T)
        deltaxb = SharedArray{Float64}(p_num, n)
        deltayb = SharedArray{Float64}(p_num, n)
        deltaxb .= 0.0
        deltayb .= 0.0
        return deltaxb, deltayb
    end
    
    # --- LGビーム関連関数 ---
    function Rayleigh_Range(w0, lambda)
        return pi * w0^2 / lambda
    end

    function Laguerre_L(p::Int, l::Int, x::Real)
        l_non_negative = abs(l)
        L_poly = SpecialPolynomials.basis(SpecialPolynomials.Laguerre{Float64(l_non_negative)}, p)
        return L_poly(x)
    end

    # !! 修正: 絶対強度を計算する関数に変更 !!
    # 参考文献: Trapping forces in optical tweezers (J. E. Curtis et al.) を基に、全パワー P を考慮した強度 I(r, z) を計算
    function LG_Intensity_Absolute(r, z, p::Int, l::Int, w0, lambda, ls_pow)
        zR = Rayleigh_Range(w0, lambda)
        current_wz = w0 * sqrt(1.0 + (z / zR)^2)
        w0_ratio = (w0 / current_wz)^2
        
        # 動径項の変数
        x_arg = 2.0 * r^2 / current_wz^2
        l_abs = abs(l)
        
        # 強度プロファイルの動径部分
        radial_term = x_arg^l_abs * (Laguerre_L(p, l, x_arg))^2 * exp(-x_arg)
        
        # LG01の場合の絶対強度 $I(r, 0) = \frac{4 P}{\pi w_0^2} \cdot \frac{r^2}{w_0^2} \cdot e^{-2r^2/w_0^2}$ を使用
        if p==0 && l==1 # LG01の絶対強度 (z=0)
            # 動径項 r/w0^2 以外 (定数部分): 4 * P / (pi * w0^2)
            scaling_factor = 4.0 * ls_pow / (pi * w0^2)
            
            # r^2/w0^2 * exp(-2r^2/w0^2) の部分
            r_wz_ratio = r^2 / current_wz^2
            intensity_absolute = scaling_factor * w0_ratio * r_wz_ratio * exp(-2.0 * r_wz_ratio)
            return intensity_absolute
        else
            # 一般的なLGモードの規格化は複雑なので、一旦LG01に限定し、それ以外はエラー
            error("Error: Only LG(0, 1) mode is currently supported for absolute intensity calculation.")
            return 0.0
        end
    end


    function particle_polarizability(a, np, nm)
        epsilon_m = nm^2
        return 4.0 * pi * epsilon_m * a^3 * ((np^2 - nm^2) / (np^2 + 2.0 * nm^2))
    end
    
    # !! 修正: 強度計算を絶対強度に基づくものに変更 !!
    function LG_Gradient_Force(x_pos, y_pos, z_pos, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        r = sqrt(x_pos^2 + y_pos^2)
        phi = atan(y_pos, x_pos)
        alpha_particle = particle_polarizability(r_p, np, nm)
        
        dr_step = 1.0e-10
        
        # 絶対強度Iを計算
        I_current = LG_Intensity_Absolute(r, z_pos, p_idx, l_idx, w0_beam, w_l, ls_pow)
        
        r_plus_dr = r + dr_step
        
        I_r_plus_dr = LG_Intensity_Absolute(r_plus_dr, z_pos, p_idx, l_idx, w0_beam, w_l, ls_pow)
        dI_dr = (I_r_plus_dr - I_current) / dr_step

        # 勾配力 $F = \frac{\alpha}{n_m c} \nabla I$ を採用
        F_r = alpha_particle * dI_dr / (nm * c) 

        Fx = F_r * cos(phi)
        Fy = F_r * sin(phi)

        # デバッグログは削除
        
        return Fx, Fy, 0.0
    end
    # --- ---

    # --- 追加: ランダムな初期位置を生成する関数 ---
    function generate_initial_position(p_num, w0_beam, ls_pow, np, nm, w_l)
        r_max = w0_beam / sqrt(2.0)  # LG01モードの強度極大の半径 (z=0)
        r = zeros(p_num)
        theta = zeros(p_num)

        # 初期位置の動径距離 r を、半径 r_max の円盤内で均一にランダム化
        for i in 1:p_num
            # r^2 が [0, r_max^2] で均一になるように (面積に比例するランダムサンプリング)
            r[i] = sqrt(rand()) * r_max
            theta[i] = 2 * pi * rand()  # 角度は [0, 2π] の範囲でランダム
        end

        # x, y の位置を決定
        x0 = r .* cos.(theta)
        y0 = r .* sin.(theta)

        return x0, y0
    end
    # --- ---


    #calculate a particle behavior
    # 引数に p_idx, l_idx, w0_beam, w_l を追加
    function euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        x_i = zeros((n+1)) #particle x-position
        y_i = zeros((n+1)) #particle y-position
        x_i[1] = x0[j, 1]#set a initial x-position
        y_i[1] = y0[j, 1]#set a initial y-position
        # 熱運動項はゼロ

        gammaf = gamma(Tave, r_p)
        
        #Let's calculate a particle behavior
        for i in 1:n
            # LGビームの勾配力を計算
            Fx_grad, Fy_grad, _ = LG_Gradient_Force(x_i[i] - ls_spot_x, y_i[i] - ls_spot_y, 0.0, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)

            # 位置更新: 熱運動の項を削除
            x_i[i+1] = x_i[i] + Fx_grad * deltat / gammaf
            y_i[i+1] = y_i[i] + Fy_grad * deltat / gammaf

            # 巨大な値への発散を検知するためのガード
            if abs(x_i[i+1]) > 100e-6 || abs(y_i[i+1]) > 100e-6 # 100 um を超えたら停止
                # デバッグログは削除
                x_i[i+1:end] .= x_i[i+1] # 残りのステップを最終位置で埋める
                y_i[i+1:end] .= y_i[i+1]
                break
            end
        end
        return x_i, y_i
    end
    
    #define parallel calculations
    # 引数に LGパラメータを追加
    function euler_para(p_num, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx::Int, l_idx::Int, w_l)
        resultx = SharedArray{Float64}(p_num, n+1)
        resulty = SharedArray{Float64}(p_num, n+1)
        @sync @distributed for j = 1:p_num
            # LGパラメータを euler に渡す
            resultx[j, :], resulty[j, :] = euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        end

        if output == "um"
            resultx = resultx .* 1e6
            resulty = resulty .* 1e6
        end
        
        # LGパラメータを含むファイル名に修正
        power_mW = Int(round(ls_pow * 1000))
        radius_nm = Int(round(r_p * 1e9))
        filename_base = "LG_p$(p_idx)_l$(l_idx)_$(power_mW)mW_$(radius_nm)nm_noBM_single" # single を追加
        
        cd("$(save_path)")
        writedlm("x_$(filename_base).txt", resultx, "\t")
        writedlm("y_$(filename_base).txt", resulty, "\t")
    end

    #main
    # 引数に LGパラメータを追加
   function main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_idx::Int, l_idx::Int, w_l)
        # generate displacements of Brrownian motion -> BM関数は空の行列を返す
        Random.seed!(seed_num)
        xb, yb = BM(p_num, Tave)
        
        # --- 修正箇所: 初期位置をランダムに設定 (generate_initial_position関数を呼び出す) ---
        # 粒子を光渦リングの強度極大半径 r_max の内側からランダムに配置する。
        x0_vec, y0_vec = generate_initial_position(p_num, w0_beam, ls_pow, np, nm, w_l)
        
        # euler_para 関数に渡すため、ベクトルを (p_num, 1) 行列に変換
        x0 = reshape(x0_vec, p_num, 1)
        y0 = reshape(y0_vec, p_num, 1)
        # ---------------------------------------------------------------------

        euler_para(p_num, n, x0, y0, xb, yb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
    end
end

# --- メイン実行 ---
# LGビームのパラメータを引数として渡す
@time main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_index, l_index, w_l)

# プロセスを終了
#2025/11/20完成版

 28.915964 seconds (7.61 M allocations: 1.155 GiB, 1.65% gc time, 0.46% compilation time)
